In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup
from glob import glob

In [4]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(file, minlen=2):
    raw_xml = parser.from_file(file, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [6]:
files = glob('govdocs/*')
len(files)

1277

In [8]:
with open('govdocs.jsonl', 'w') as fopen:
    for f in tqdm(files):
        t = '\n'.join(parse_tika(f))
        data = {
            'file': f,
            'text': t,
        }
        fopen.write(f'{json.dumps(data)}\n')

100%|███████████████████████████████████████| 1277/1277 [03:50<00:00,  5.54it/s]
